# This Utilizes Tkinter for consumer GUI

In [13]:
# Imports
import sys
import tkinter as tk
from tkinter import filedialog, messagebox

from components.ui_controller import SQLConverterUI
from components.sql_parser import SQLConverter
from components.data_cleaner import SQLCleaner
from components.file_handler import FileHandler

In [17]:
# Seperate handlers class to handle operations on ApplicationController
class handlers:
    def __init__(self, controller):
        self.controller = controller

    def browse_file(self):
        # Open dialog to select SQL file (Allow .txt in the event the user copied and pasted SQL from Tableau to a txt)
        file_path = filedialog.askopenfilename(
            title="Select Tableau SQL File",
            filetypes=[
                ("SQL Files", "*.sql"),
                ("Text Files", "*.txt"),
                ("All Files", "*.*")
            ]
        )
        
        if file_path:
            self.controller.current_file_path = file_path
            self.controller.ui.set_file_path(file_path)
            
            # Provide file info, to confirm the file selected
            file_info = self.controller.file_handler.get_file_info(file_path)
            if file_info:
                info_text = f"File: {file_info['filename']} | Size: {file_info['size_kb']:.2f} KB"
                self.controller.ui.set_file_info(info_text)
            
            self.controller.ui.set_status(f"File selected: {file_info['filename']}")



    def load_and_convert(self):
        '''
        Load and convert the selected file
        '''
        # Check if a valid file is selected
        if not self.controller.current_file_path:
            self.controller.ui.show_warning("No File", "Please select a SQL file first.")
            return
        
        # Validate file
        is_valid, error = self.controller.file_handler.validate_file(self.controller.current_file_path)
        if not is_valid:
            self.controller.ui.show_error("Invalid File", error)
            return
        
        # Use try for any errors not considered (There are a lot of commadns and new releases)
        try:
            # Read file using file_handler.py
            raw_sql = self.controller.file_handler.read_file(self.controller.current_file_path)
            
            # Clean and prepare SQL using data_cleaner.py
            prepared_data = self.controller.cleaner.prepare_for_parsing(raw_sql)
            self.controller.tableau_sql = prepared_data['cleaned_query']
            
            # Update UI with Tableau SQL (User's Input)
            self.controller.ui.set_tableau_sql(self.controller.tableau_sql)
            
            # Convert SQL using sql_parser.py
            self.controller.fabric_sql, self.controller.current_metrics, self.controller.flagged_items = self.controller.converter.convert_query(self.controller.tableau_sql)
            
            # Display Fabric SQL
            self.controller.ui.set_fabric_sql(self.controller.fabric_sql)
            
            # Display flagged items
            '''
            TODO Update the flagging system, to be more simple, there is some sort of uncaught exception 
            I think the issue is the line wrapping, when several lines can be condensed into one by Fabric and vice versa for longer statements
            '''
            self.controller.ui.display_flagged_items(self.controller.flagged_items, self.controller.current_metrics)
            
            # Enable save button
            self.controller.ui.enable_save_button()
            

            # TODO: Removed update status as its not correctly updating, fix this later (Currently disabled)
            # Update status
            metrics = self.controller.current_metrics.to_dict()
            # success_rate = metrics['success_rate']
            # status_text = f"Conversion complete! Success rate: {success_rate:.1f}%"
            
            if self.controller.flagged_items:
                status_text = f" | {len(self.controller.flagged_items)} item(s) flagged for review"
            
            self.controller.ui.set_status(status_text)
            
            # Show completion message
            self.controller.ui.show_info("Success", "SQL conversion completed successfully!")

        except Exception as e:
            self.controller.ui.show_error("There was an issue with conversion", f"An error occurred during conversion:\n{str(e)}")

    def save_converted_sql(self):
        """Save the converted Fabric SQL to a file."""
        current_fabric_sql = self.controller.ui.get_fabric_sql()
        
        # If file was unable to convert {incorrect file type or syntax, or just empty file}
        if not current_fabric_sql:
            self.controller.ui.show_warning("No Data", "No converted SQL to save.")
            return
        
        # If file was able to convert, then save it
        if self.controller.current_file_path:
            default_path = self.controller.file_handler.generate_output_filename(self.controller.current_file_path)
        else:
            default_path = "converted_fabric.sql"
        
        save_path = filedialog.asksaveasfilename(
            title="Save SQL Generated File",
            initialfile=default_path,
            defaultextension=".sql",
            filetypes=[("SQL Files", "*.sql"), ("Text Files", "*.txt"), ("All Files", "*.*")]
        )
        
        if save_path:
            try:
                self.controller.file_handler.write_file(save_path, current_fabric_sql)
                self.controller.ui.show_info("Success", f"Converted SQL saved to:\n{save_path}")
                self.controller.ui.set_status(f"File saved successfully: {save_path}")
            except Exception as e:
                self.controller.ui.show_error("Save Error", f"Failed to save file:\n{str(e)}")

    def clear_all(self):
        """Reset application"""
        self.controller.current_file_path = None
        self.controller.tableau_sql = ""
        self.controller.fabric_sql = ""
        self.controller.current_metrics = None
        self.controller.flagged_items = []
        self.controller.ui.clear_all()

In [18]:
class ApplicationController:
    def __init__(self):
        self.root = tk.Tk()
        
        # Initialize logic components
        self.converter = SQLConverter()
        self.cleaner = SQLCleaner()
        self.file_handler = FileHandler()
        
        # App state
        self.current_file_path = None
        self.tableau_sql = ""
        self.fabric_sql = ""
        self.current_metrics = None
        self.flagged_items = []
        
        # Initialize handlers, passing SELF (this controller instance)
        self.handlers = handlers(self)
        
        # Callbacks point to the handler instance's methods
        callbacks = {
            'on_browse': self.handlers.browse_file,
            'on_convert': self.handlers.load_and_convert,
            'on_save': self.handlers.save_converted_sql,
            'on_clear': self.handlers.clear_all
        }
        self.ui = SQLConverterUI(self.root, callbacks)

    def run(self):
        try:
            self.ui.run()
        except Exception as e:
            print(f"Application error: {e}")


In [19]:
def main():
    app = ApplicationController()
    app.run()

if __name__ == "__main__":
    main()